In [ ]:
import cv2
import numpy as np
import os
import random
import torch
import config
import vtk

from model_structure_3d import UNet3D
from tqdm import tqdm
from copy import deepcopy as copy

In [ ]:
# Load the PyTorch model
model = UNet3D()
model.load_state_dict(torch.load("./model_for_vasc_3d.pth", map_location="cpu"))

# Set the model to evaluation mode
model.eval()

# Directory paths for input and output data
input_dir = "./preparations/preprocess/indata"
output_dir = "./preparations/preprocess/outdata"

# Get a list of image files in the input directory
image_files = os.listdir(input_dir)

In [ ]:
start_index = random.randint(0, len(image_files) // config.NUM_PICS - 1) * 16

# Choose 16 consecutive images starting from the random index
label_image = cv2.imread(os.path.join(output_dir, image_files[0]), cv2.IMREAD_GRAYSCALE)
shape = label_image.shape

while True:
    random_x, random_y = random.randint(0, shape[0] - config.WIDTH - 1), random.randint(0, shape[1] - config.HEIGHT - 1)
    # Initialize an empty list to store the images
    image_list = []
    label_list = []
    for i in range(start_index, start_index + 16):
        # Construct the full file paths
        input_image_path = os.path.join(input_dir, image_files[i])
        output_label_path = os.path.join(output_dir, image_files[i])

        label_image = cv2.imread(output_label_path, cv2.IMREAD_GRAYSCALE)

        label_image = label_image[random_y : random_y + config.HEIGHT, random_x : random_x + config.WIDTH]
        img = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)[random_y : random_y + config.HEIGHT, random_x : random_x + config.WIDTH]
        mask = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
        input_image = img * (mask // 255)

        # Append the processed image to the list
        image_list.append(input_image)
        label_list.append(label_image)

    # Convert the list of images into a NumPy array
    if(np.sum(label_list) > 0):
        break

image_array = np.array([image_list])
label_list = np.array(label_list)

print(image_array.shape)

# Perform inference using the model
with torch.no_grad():
    input_tensor = torch.from_numpy(image_array / 255).unsqueeze(0).float()
    output = model(input_tensor)
# Convert the output tensor to a NumPy array
output_np = output.numpy()
# Convert the output prediction to binary format and multiply by 255
binary_output = (output_np >= 0.5).astype(np.uint8)[0] * 255
# Display the input image, label, and binary prediction using OpenCV

img_in = ((np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) * 255)[0]
print(img_in.shape)
new_img = np.array([cv2.resize(x, (512, 512)) for x in img_in])

# Define video properties
height, width = new_img.shape[1:]
channels = 1
fps = 3
video_duration = len(new_img) / fps

# Create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, fps, (width, height))
num = 0
# Display the frames in OpenCV window
for i in range(len(new_img)):
    # Convert the frame to uint8 if necessary
    new_img[i] = np.uint8(new_img[i])
    
    # Write the frame to the video file
    out.write(new_img[i])

    # Display the frame in OpenCV window (uncomment if needed)
    img_concat = np.concatenate((new_img[i], cv2.resize(image_array[0][i], (512, 512))), axis=1)
    cv2.imshow('Frame' + str(num), img_concat)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    num += 1

# Release the VideoWriter and close the OpenCV window if used
out.release()

print(new_img.shape)
# cv2.imshow("Input Image", cv2.resize(image_array, (512, 512)))
# cv2.imshow("Label Image", cv2.resize(label_list, (512, 512)))
# cv2.imshow("Model Prediction (Binary)", cv2.resize(binary_output, (512, 512)))
# cv2.imshow("Correctness Image", cv2.resize((np.multiply(binary_output, label_image) + np.multiply((255 - binary_output), (255 - label_image))) * 255, (512, 512)))

TP = np.sum(np.multiply(binary_output, label_image))
TN = np.sum(np.multiply((255 - binary_output), (255 - label_image)))
FP = np.sum(np.multiply(binary_output, (255 - label_image)))
FN = np.sum(np.multiply((255 - binary_output), label_image))
print("TP:", TP)
print("TN:", TN)
print("FP:", FP)
print("FN:", FN, "\n")

print("Positive accuracy:", np.round(TP / (TP + FN), 3))
print("Negative accuracy:", np.round(TN / (TN + FP), 3))
print("Accuracy:", np.round((TP + TN) / (TP + TN + FP + FN), 3))

print("")
print("Positive Recall:", np.round(TP / (TP + FP), 3))
print("Negative Recall:", np.round(TN / (TN + FN), 3))
print("Recall:", np.round((TP + TN) / (TP + TN + FP + FN), 3))
# Wait for a key press and then close the OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def get_prediction(img, margin):
    if margin != 0:
        raise ValueError
    with torch.no_grad():
        input_tensor = torch.from_numpy(img / 255).unsqueeze(0).float()
        output = model(input_tensor)
    output_np = output.numpy()
    return output_np

input_dir = "./preparations/data/indata"
output_dir = "./preparations/data/outdata"

margin = 0
step = 64
# Get a list of image files in the input directory
image_files = os.listdir(input_dir)

show_size = (512, 512)

random_image_file_index = random.randint(0, len(image_files) - config.NUM_PICS)
print("Random image name:", image_files[random_image_file_index])
# random_image_file = "img_1000342.png"
input_image_path = [os.path.join(input_dir, image_files[i]) for i in range(random_image_file_index, random_image_file_index + config.NUM_PICS)]
output_label_path = [os.path.join(output_dir, image_files[i]) for i in range(random_image_file_index, random_image_file_index + config.NUM_PICS)]

input_images = []
label_images = []
for i in range(config.NUM_PICS):
    input_image_tmp = cv2.imread(input_image_path[i], cv2.IMREAD_GRAYSCALE)
    _, label_image_tmp = cv2.threshold(input_image_tmp, 61, 255, cv2.THRESH_BINARY)
    input_images.append(input_image_tmp)
    label_images.append(label_image_tmp)

input_images = np.array([input_images])
label_images = np.array([label_images])

shape = label_images.shape
print(shape)

input_images = input_images[:, :, shape[0] % step:, shape[1]%step:]
label_images = label_images[:, :, shape[0] % step:, shape[1]%step:]

shape = label_images.shape

count_times = np.zeros(shape)
total_times = np.zeros(shape)
print("Shape between loop:", input_images.shape)
arr1 = []
arr2 = []
for i in tqdm(range(0, shape[2] - shape[2] % step - config.HEIGHT, step), "Processing"):
    for j in range(0, shape[3] - shape[3] % step - config.WIDTH, step):
        slice_tmp = input_images[:, :, i : i + config.HEIGHT, j : j + config.WIDTH]
        array_tmp = get_prediction(slice_tmp, margin)[0]
        count_times[:, :, i : (i + config.HEIGHT), j : (j + config.WIDTH)] += (array_tmp > 0.5)
        total_times[:, :, i : (i + config.HEIGHT), j : (j + config.WIDTH)] += 1
      

confidence = (count_times + 0.0001) / (total_times + 0.0001)
binary_output = (np.floor(confidence * 255)).astype(np.uint8)

label_images = label_images[0]
binary_output = binary_output[0]

shape = label_images.shape
# Create a new array with default values (127, 127, 255)
result_array = np.zeros(shape + (3,), dtype=np.uint8)

# Update values based on conditions

result_array[(label_images == 0) & (binary_output == 0), :] = (0, 0, 0)  # Where both are 0
result_array[(label_images == 255) & (binary_output == 255), :] = (255, 255, 255)  # Where both are 255
result_array[(label_images == 0) & (binary_output == 255), :] = (0, 0, 255)  # Where label_image is 0 and binary_output is 255
result_array[(label_images == 255) & (binary_output == 0), :] = (0, 128, 128)  # Where label_image is 255 and binary_output is 0

a, b, c, d = result_array.shape

# Calculate the number of rows and columns to keep
new_b = b - (b % step)
new_c = c - (c % step)

# Create copies for later
label_image_copy = label_images[:, :new_b, :new_c]
binary_output_copy = binary_output[:, :new_b, :new_c]
# Use array slicing to obtain the desired subarray
coef = 0.5
input_images = input_images[0]


input_images = np.repeat(input_images[:, :new_b, :new_c, np.newaxis], 3, axis=-1)
label_images = np.repeat(label_images[:, :new_b, :new_c, np.newaxis], 3, axis=-1)
binary_output = np.repeat(binary_output[:, :new_b, :new_c, np.newaxis], 3, axis=-1)
result_array = result_array[:, :new_b, :new_c, :]


modified_label_images = label_images.copy()
COLOR_BG = [1, 1, 1] #BGR WHITE
COLOR_BODY = [0, 0, 0] #BGR BLUE

modified_label_images[np.all(label_images == [0, 0, 0], axis=-1)] = COLOR_BG
modified_label_images[np.all(label_images == [255, 255, 255], axis=-1)] = COLOR_BODY

blend_images = input_images * modified_label_images

# Specify the text, font, and position
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
font_thickness = 2
text_color = (255, 255, 255)  # White color
size_x = input_images.shape[1] // 2
size_y = input_images.shape[2] // 2
text_position_x = 10
text_position_y = 25

for i in range(config.NUM_PICS):

    img_vert1 = np.concatenate([cv2.resize(input_images[i], show_size), cv2.resize(label_images[i], show_size)], axis=0)
    img_vert2 = np.concatenate([cv2.resize(blend_images[i], show_size), cv2.resize(binary_output[i], show_size)], axis=0).astype(np.uint8)

    img_collage = np.concatenate([img_vert1, img_vert2], axis=1)

    # Use cv2.putText() to overlay text on the image
    cv2.putText(img_collage, "Input", (text_position_x, text_position_y), font, font_scale, text_color, font_thickness)
    cv2.putText(img_collage, "Label", (text_position_x, text_position_y + show_size[1]), font, font_scale, text_color, font_thickness)
    cv2.putText(img_collage, "Prediction", (text_position_x + show_size[0], text_position_y  + show_size[1]), font, font_scale, text_color, font_thickness)
    cv2.putText(img_collage, "Blend", (text_position_x + show_size[0], text_position_y), font, font_scale, text_color, font_thickness)

    cv2.imshow("Input/Output - Label/LabelCorr - Frame " + str(i), img_collage)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# cv2.imwrite("./gener_image.png", result_array)

TP = np.count_nonzero(np.multiply(binary_output_copy, label_image_copy))
TN = np.count_nonzero(np.multiply((255 - binary_output_copy), (255 - label_image_copy)))
FP = np.count_nonzero(np.multiply(binary_output_copy, (255 - label_image_copy)))
FN = np.count_nonzero(np.multiply((255 - binary_output_copy), label_image_copy))
print("TP:", np.round(TP / (TP + TN + FP + FN) * 100, 2), "%")
print("TN:", np.round(TN / (TP + TN + FP + FN) * 100, 2), "%")
print("FP:", np.round(FP / (TP + TN + FP + FN) * 100, 2), "%")
print("FN:", np.round(FN / (TP + TN + FP + FN) * 100, 2), "%")

print()
print("Positive accuracy:", TP / (TP + FN))
print("Negative accuracy:", TN / (TN + FP))
print("Total accuracy:", (TP + TN) / (TP + TN + FP + FN))

print("")
print("Positive Recall:", TP / (TP + FP))
print("Negative Recall:", TN / (TN + FN))

# Wait for a key press and then close the OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()

for x in result_array:
    img_tmp = x
    print(img_tmp.shape)
    cv2.putText(img_tmp, "Yellow - FN", (text_position_x, text_position_y), font, font_scale, text_color, font_thickness)
    cv2.putText(img_tmp, "Red - FP", (text_position_x, 2 * text_position_y), font, font_scale, text_color, font_thickness)
    cv2.imshow("Correctness Image", img_tmp.astype(np.uint8))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# cv2.putText(result_array, "Yellow - FP", (text_position_x, text_position_y), font, font_scale, text_color, font_thickness)
# cv2.putText(result_array, "Red - FN", (text_position_x, 2 * text_position_y), font, font_scale, text_color, font_thickness)
# cv2.imshow("Correctness Image", result_array)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [ ]:
# input_dir = "./preparations/data/indata"
# output_dir = "./preparations/preprocess/"
# # Create the folder if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)
# os.makedirs(os.path.join(output_dir, "indata"), exist_ok=True)
# os.makedirs(os.path.join(output_dir, "outdata"), exist_ok=True)

# for image_file in tqdm(os.listdir(input_dir)):
#     input_image_path = os.path.join(input_dir, image_file)
#     output_image_path1 = os.path.join(output_dir, "indata", image_file)
#     output_image_path2 = os.path.join(output_dir, "outdata", image_file)
#     img = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)
#     mask = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
#     output_image = img * (mask // 255)
#     cv2.imwrite(output_image_path1, output_image)
#     _, thresh = cv2.threshold(output_image, 61, 255, cv2.THRESH_BINARY)
#     cv2.imwrite(output_image_path2, thresh)

In [ ]:
# import cv2
# import os
# import nibabel as nib
# import numpy as np
# from tqdm import tqdm

# def nii_to_numpy(file_path):
#     # Load NRRD file
#     nrrd_data = nib.load(file_path)

#     # Get the data array from the NRRD file
#     nrrd_array = nrrd_data.get_fdata()

#     return nrrd_array

# # Example usage
# nrrd_file_path = "./single_image.nii"
# numpy_array = np.squeeze(nii_to_numpy(nrrd_file_path), axis=-1)

# img_name = "img_1000342.png"
# img1 = cv2.imread(os.path.join("./preparations/data/indata/", img_name), cv2.IMREAD_GRAYSCALE)
# img2 = cv2.imread(os.path.join("./preparations/data/outdata", img_name), cv2.IMREAD_GRAYSCALE)
# img3 = np.transpose(numpy_array * 255)

# size = (512, 512)
# min_err = 2e-10
# min_err_threshold = 256

# weight = 100


# _, img_threshold = cv2.threshold(img1, 61, 255, cv2.THRESH_BINARY)
# cv2.imshow("Original", cv2.resize(img1, size))
# cv2.imshow("Threshold", cv2.resize(img2, size))
# cv2.imshow("Manual", cv2.resize(img3, size))
# cv2.imshow("Closest", cv2.resize(img_threshold, size))
# cv2.waitKey()
# cv2.destroyAllWindows()

# # Create a new array with default values (255, 0, 0)
# result_array = np.full((img3.shape[0], img3.shape[1], 3), (0, 0, 255), dtype=np.uint8)

# # Update values based on conditions
# result_array[(img3 == 0) & (img_threshold == 0), :] = (0, 0, 0)  # Where both are 0
# result_array[(img3 == 255) & (img_threshold == 255), :] = (255, 255, 255)  # Where both are 255


# cv2.imshow("Errors", result_array)
# cv2.imwrite("./image_with_errors.png", result_array)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [ ]:
# orig_data = os.listdir("Z://Artem/orig_data/")
# annotated_data = os.listdir("Z://Artem/annotated_data/")
# pic_in = cv2.imread("Z://Artem/orig_data/img_1000000.png", cv2.IMREAD_GRAYSCALE)
# pic_out = cv2.imread("Z://Artem/annotated_data/img_1000000.png", cv2.IMREAD_GRAYSCALE)
# pic_pred = cv2.imread("Z://Artem/CNN_data/img_1000000.png", cv2.IMREAD_GRAYSCALE)
# print(pic_in.shape)
# print(pic_out.shape)
# print(pic_pred.shape)

# orig_data = os.listdir("Z://Artem/orig_data/")
# annotated_data = os.listdir("Z://Artem/annotated_data/")

# step = 32

# # for x in tqdm(orig_data, ""):
# #     file_path = os.path.join("Z://Artem/orig_data/", x)
# #     pic = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
# #     height, width = pic.shape
# #     cv2.imwrite(file_path, pic[:height- height % step, :width - width % step])

# for x in tqdm(annotated_data, ""):
#     file_path = os.path.join("Z://Artem/annotated_data/", x)
#     pic = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
#     height, width = pic.shape
#     cv2.imwrite(file_path, pic[:height - height % step, :width - width % step])
